# Experimenting with clustering the data and fitting XGBoost model to each cluster
This notebook is part of my Mercedes-Benz Kaggle efforts. Competion @: https://www.kaggle.com/c/mercedes-benz-greener-manufacturing Use of data subject to conditions mentioned in above link.
## With additional data generation:
+ additional data will be using mean with extremes removed (not min and max as in part 1)<br>
+ additional data will be used for training the models, while supplied training data will be used for evaluatig (testing) of the models.<br>

## With clustering of the data:
+ use Birch Clustering algorithm, 3, 4, 5, 25 clusters.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import string
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import r2_score
from sklearn import metrics
#from sklearn.gaussian_process.kernels import Matern
from sklearn.cluster import Birch

/home/kaggler/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## 1. Load Data Sets

In [17]:
df_train = pd.read_csv('../data/train.csv', index_col = 'ID').fillna(0)
df_train = df_train[df_train['y'] < 200] # drop one outlier data
df_test = pd.read_csv('../data/test.csv', index_col = 'ID').fillna(0)
li_del_columns = []

## 2. Construct additional features and change letter categorical to binary

In [18]:
def my_dummies(c, df1, df2):
    l = len(df1)
    df = pd.concat([df1, df2], axis = 0)
    df_temp = pd.get_dummies(df[c], prefix=c)
    df = pd.concat([df_temp, df], axis = 1)
    df = df.drop(c, axis = 1)
    df1 = df[ : l].copy()
    df2 = df[l : ].copy()
    return df1, df2

In [19]:
li_columnNames = [e for e in list(df_train.columns.values) if e not in {'y', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8'}]
li_categorical = ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']
list_value = list(string.lowercase) + list('a' + x for x in list(string.lowercase)) + ['ba', 'bb', 'bc']

df_train_eng = df_train.copy()
df_test_eng = df_test.copy()
df_train_eng['qty_options'] = df_train_eng[li_columnNames].sum(axis=1)
df_test_eng['qty_options'] = df_test_eng[li_columnNames].sum(axis=1)

for c in li_categorical:
    df_train_eng, df_test_eng = my_dummies(c, df_train_eng, df_test_eng)


## 3. some data clensing / Identifying data for possible clensing
### Identify duplicate columns

In [20]:
all_data = pd.concat((df_train_eng, df_test_eng))

# remove duplicated columns
c = all_data.columns
print('\n Number of columns before cleaning: %d' % len(c))
li_duplicates = []
for i in range(len(c)-1):
    v = all_data[c[i]].values
    for j in range(i+1,len(c)):
        if np.array_equal(v,all_data[c[j]].values):
            li_duplicates.append(c[j])
            print(' Column %s is identical to %s. Removing %s' % (str(c[i]), str(c[j]), str(c[j])))
li_duplicates = list(set(li_duplicates))
print '\n Number of identified duplicates marked: %s' % len(li_duplicates)


 Number of columns before cleaning: 581
 Column X2_a is identical to X32. Removing X32
 Column X2_ac is identical to X125. Removing X125
 Column X2_ac is identical to X227. Removing X227
 Column X2_ad is identical to X107. Removing X107
 Column X2_ag is identical to X30. Removing X30
 Column X2_ai is identical to X215. Removing X215
 Column X2_am is identical to X260. Removing X260
 Column X2_ap is identical to X16. Removing X16
 Column X2_aq is identical to X211. Removing X211
 Column X2_as is identical to X251. Removing X251
 Column X2_at is identical to X184. Removing X184
 Column X2_at is identical to X262. Removing X262
 Column X2_au is identical to X59. Removing X59
 Column X2_aw is identical to X213. Removing X213
 Column X2_aw is identical to X67. Removing X67
 Column X2_b is identical to X26. Removing X26
 Column X2_d is identical to X97. Removing X97
 Column X2_f is identical to X23. Removing X23
 Column X2_g is identical to X112. Removing X112
 Column X2_g is identical to X

### Dropping duplicate columns
(yes there are columns duplicate and complimentary, so need to do it twice)

In [21]:
print 'Number columns before cleaning: ', df_train_eng.shape[1],
l = len(df_train_eng)
df = pd.concat([df_train_eng, df_test_eng], axis = 0)
for c in li_duplicates:
    df.drop(c, axis=1, inplace = True)

df_train_eng = df[ : l].copy()
df_test_eng = df[l : ].copy()
print 'and after cleaning: ', df_train_eng.shape[1]

Number columns before cleaning:  581 and after cleaning:  519


### Identify complementary columns

In [22]:
list_index = [e for e in list(df_train_eng.columns.values) if e not in {'y_actual', 'y', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8'}]
all_data = pd.concat((df_train_eng[list_index], df_test_eng[list_index]))
value_compliment = {0: 1, 1: 0}

# remove complimentary columns
c = all_data.columns
print('\n Number of columns before cleaning: %d' % len(c))
li_compliments = []
for i in range(len(c)-1):
    v = all_data[c[i]].replace(value_compliment).values
    for j in range(i+1,len(c)):
        if np.array_equal(v,all_data[c[j]].values):
            li_compliments.append(c[j])
            print(' Column %s is complimentary to %s. Removing %s' % (str(c[i]), str(c[j]), str(c[j])))
li_compliments = list(set(li_compliments))
print '\n Number of identified complements marked: %s' % len(li_compliments)


 Number of columns before cleaning: 518
 Column X128 is complimentary to X130. Removing X130
 Column X156 is complimentary to X157. Removing X157
 Column X204 is complimentary to X205. Removing X205
 Column X232 is complimentary to X263. Removing X263

 Number of identified complements marked: 4


### Droping complementary columns

In [23]:
print 'Number columns before cleaning: ', df_train_eng.shape[1],
l = len(df_train_eng)
df = pd.concat([df_train_eng, df_test_eng], axis = 0)
for c in li_compliments:
    df.drop(c, axis=1, inplace = True)

df_train_eng = df[ : l].copy()
df_test_eng = df[l : ].copy()
print 'and after cleaning: ', df_train_eng.shape[1]

Number columns before cleaning:  519 and after cleaning:  515


### let's see if we have same datarows (other then y and y_actual) in train data
<font color=red>SKIP TO "GENERATING MORE DATA" IF LOADING PRE-PROCESSED DATASET. THIS STEP TAKES TIME!</font>

<i>mean</i> is calcualted out of all identical datarows.<br>
<i>mean minus extremes</i> is calculated if four or more datasets are identical by removing the lowest and highest value. <br>
<i>mean narrowing</i> is calculated if four or more datasets are identical by removing values that are more than 5% away from mean

In [ ]:
li_columns = [e for e in list(df_train_eng.columns.values) if e not in {'y_actual', 'y'}]
df = df_train_eng[li_columns]
li_seen_duplicates = []
li_duplicates = []
print 'start finding...'
#for i in range(235):
for i in range(len(df)):
    if i not in li_seen_duplicates:
        set_pairs = {}
        v = df.iloc[i, :].values
        for j in range(i+1,len(df)):
            if np.array_equal(v,df.iloc[j, :].values):
                if i not in li_seen_duplicates: li_seen_duplicates.append(i)
                li_seen_duplicates.append(j)
                if i not in set_pairs: set_pairs[i] = df_train_eng.ix[df_train_eng.index.tolist()[i],'y']
                set_pairs[j] = df_train_eng.ix[df_train_eng.index.tolist()[j],'y']
        if set_pairs: li_duplicates.append(set_pairs)
    if ((i > 0) & (i % 100 == 0)): print '.',
for i in range(len(li_duplicates)):
    values = li_duplicates[i].values()
    val_narrow = []
    mean = sum(values) / len(values)
    deviate = 0.08
    print '\n Row IDs ', list(li_duplicates[i].keys()), ' mean: ', mean, ' with values: ', values,
    if len(values) >= 4:
        print ', mean minus extremes: ', (sum(values) - min(values) - max(values)) / (len(values) - 2),
    if len(values) >= 4:
        for v in values:
            if mean*(1-deviate) <= v <= mean*(1+deviate):
                val_narrow.append(v)
        print ', mean narrowing: ', sum(val_narrow) / len(val_narrow),

Next, let's make a copy of the trainingdata and set the values for all identical row sets to the mean or mean narrow if more than 3 values.

In [ ]:
df_train_cp = df_train_eng.copy()

In [ ]:
for i in range(len(li_duplicates)):
    values = li_duplicates[i].values()
    mean = sum(values) / len(values)
    if len(values) > 3:
        for v in values:
            if mean*(1-deviate) <= v <= mean*(1+deviate):
                val_narrow.append(v)
        mean = sum(val_narrow) / len(val_narrow)
    for j in list(li_duplicates[i].keys()):
        df_train_cp.set_value(j ,'y', mean)

Write out the dataframe as is to load in this state if required.

In [ ]:
df_train_cp.to_csv(path_or_buf='../data/train_cp.csv', index_label='ID', header=True)

In [ ]:
df_train_eng.to_csv(path_or_buf='../data/train_eng.21.csv', index_label='ID', header=True)

In [24]:
df_test_eng.to_csv(path_or_buf='../data/test_eng.21.csv', index_label='ID', header=True)

### generating some more training data
the amount of data considering the binary features is not much, hence let's produce some more.
I figure I'll be adding 10 rows for each pre-existing row: I'll be adding / subtracting a small percentage (probably 1% times number of rows away from start) from the goal y value. The average of the rowset stays the same; the variance is smaller to what I have seen in the dataset earlier (see checking for identical rows and the y values). This should afford me sufficient data to get good model fitting.

In [25]:
df_train_cp = pd.read_csv('../data/train_cp.csv', index_col = 'ID').fillna(0)

In [26]:
df_train_eng = pd.read_csv('../data/train_eng.19.csv', index_col='ID').fillna(0)

In [32]:
df_test_eng = pd.read_csv('../data/test_eng.21.csv', index_col='ID').fillna(0)

In [ ]:
df_add = pd.DataFrame.from_items([(name, pd.Series(data=None, dtype=series.dtype)) for name, series in df_train_cp.iteritems()])
df_add = pd.DataFrame(data = None, columns=df_train_cp.columns)
y_index = df_train_cp.columns.get_loc('y')
add_vals = 10
add_factor = 0.01
add_range = range(-(add_vals/2), 0) + range(1, (add_vals/2) + 1)
for i in range(len(df_train_cp)):
    values = list(df_train_cp.iloc[i,:])
    li_add = []
    for j in add_range:
        li_add.append(values[:])
        li_add[len(li_add)-1][y_index] = values[y_index]*(1 + j*add_factor)
    df_add = df_add.append(pd.DataFrame(li_add, columns=df_train_cp.columns))

In [ ]:
df_train_cp = pd.concat([df_train_cp, df_add], axis = 0)
df_train_cp.shape

In [ ]:
df_train_cp['y_actual'] = df_train_cp['y'] * df_train_cp['qty_options'] # Remember that one from further up? Now you see why.

In [27]:
df_train_eng['y_actual'] = df_train_eng['y'] * df_train_eng['qty_options']

In [ ]:
df_train_cp.to_csv(path_or_buf='../data/train_cp_XL.csv', index_label='ID', header=True)

In [28]:
df_train_cp = pd.read_csv('../data/train_cp_XL.csv', index_col = 'ID').fillna(0)

In [29]:
for i in ['X3_a', 'X3_b', 'X3_c', 'X3_d', 'X3_e', 'X3_f', 'X3_g']:
    print i, ' mean y:',
    print df_train_cp[(df_train_cp[i] == 1)]['y'].mean(), ' y_actual:',
    print df_train_cp[(df_train_cp[i] == 1)]['y_actual'].mean(), 'sigma:',
    print df_train_cp[(df_train_cp[i] == 1)]['y'].std(), ' Datasets:',
    print df_train_cp[(df_train_cp[i] == 1)].shape[0]

X3_a  mean y: 102.54389213  y_actual: 6252.06563224 sigma: 12.2050103031  Datasets: 4840
X3_b  mean y: 100.278157895  y_actual: 5606.79377193 sigma: 10.7172358958  Datasets: 627
X3_c  mean y: 101.991260849  y_actual: 6119.74591832 sigma: 12.7935021007  Datasets: 21362
X3_d  mean y: 104.747732493  y_actual: 5909.57985247 sigma: 10.2268751422  Datasets: 3190
X3_e  mean y: 100.079326692  y_actual: 5756.25331581 sigma: 11.0200814603  Datasets: 1793
X3_f  mean y: 96.7962847608  y_actual: 5230.18854792 sigma: 13.3507639374  Datasets: 11825
X3_g  mean y: 100.807721005  y_actual: 5902.35643765 sigma: 10.6391764704  Datasets: 2651


In [30]:
for i in ['X3_a', 'X3_b', 'X3_c', 'X3_d', 'X3_e', 'X3_f', 'X3_g']:
    print i, ' mean y:',
    print df_train_eng[(df_train_eng[i] == 1)]['y'].mean(), ' y_actual:',
    print df_train_eng[(df_train_eng[i] == 1)]['y_actual'].mean(), 'sigma:',
    print df_train_eng[(df_train_eng[i] == 1)]['y'].std(), ' Datasets:',
    print df_train_eng[(df_train_eng[i] == 1)].shape[0]

X3_a  mean y: 102.507477273  y_actual: 6251.17375 sigma: 11.7209104558  Datasets: 440
X3_b  mean y: 100.239649123  y_actual: 5609.35368421 sigma: 10.2040648352  Datasets: 57
X3_c  mean y: 101.959268795  y_actual: 6117.26341401 sigma: 12.4146420334  Datasets: 1942
X3_d  mean y: 105.114413793  y_actual: 5930.47831034 sigma: 9.87888653322  Datasets: 290
X3_e  mean y: 100.033190184  y_actual: 5751.59521472 sigma: 10.4960935535  Datasets: 163
X3_f  mean y: 96.4075255814  y_actual: 5209.7048186 sigma: 13.0698420549  Datasets: 1075
X3_g  mean y: 100.428672199  y_actual: 5882.50087137 sigma: 10.1089545237  Datasets: 241


I'll use df_train_cp as training data for the clustering and df_train_eng to test the quality of the clustering

In [46]:
X = df_train_cp.copy()
X.drop('y', axis = 1, inplace = True)
X.drop('y_actual', axis = 1, inplace = True)
train = df_train_eng.copy()
train.drop('y', axis = 1, inplace = True)
train.drop('y_actual', axis = 1, inplace = True)
test = df_test_eng.copy()
test.drop('y', axis = 1, inplace = True)
#X = StandardScaler().fit_transform(X)
li_model = []
li_cp_clusters = []
li_train_clusters = []
li_test_clusters = []

for i in [3, 4, 5, 25]:
    birch_model = Birch(threshold = 0.5, branching_factor = 20, n_clusters = i).fit(X)
    labels = birch_model.labels_
    li_cp_clusters.append(labels)
    centroids = birch_model.subcluster_centers_
    n_clusters = np.unique(labels).size
    #X[str(i)] = labels
    li_model.append(birch_model)
    li_train_clusters.append(birch_model.predict(train))
    li_test_clusters.append(birch_model.predict(test))
    
    # Number of clusters in labels, ignoring noise if present.
    lab = li_train_clusters[len(li_train_clusters)-1]
    n_clusters_ = len(set(lab)) - (1 if -1 in lab else 0)
    print('Number of clusters in train: %d' % n_clusters_), ' with',
    if -1 not in lab:
        print 'no',
    print 'noise'
    print("Silhouette Coefficient: %0.3f"
          % metrics.silhouette_score(train, lab))
    
    lab = li_test_clusters[len(li_test_clusters)-1]
    n_clusters_ = len(set(lab)) - (1 if -1 in lab else 0)
    print('Number of clusters in test: %d' % n_clusters_), ' with',
    if -1 not in lab:
        print 'no',
    print 'noise'
    print("Silhouette Coefficient: %0.3f"
          % metrics.silhouette_score(test, lab))

Number of clusters in train: 3  with no noise
Silhouette Coefficient: 0.249
Number of clusters in test: 3  with no noise
Silhouette Coefficient: 0.242
Number of clusters in train: 4  with no noise
Silhouette Coefficient: 0.207
Number of clusters in test: 4  with no noise
Silhouette Coefficient: 0.202
Number of clusters in train: 5  with no noise
Silhouette Coefficient: 0.151
Number of clusters in test: 5  with no noise
Silhouette Coefficient: 0.146
Number of clusters in train: 24  with no noise
Silhouette Coefficient: 0.132
Number of clusters in test: 24  with no noise
Silhouette Coefficient: 0.123


In [47]:
#for i in range(len(li_train_clusters)):
for i in range(2):
    for j in range(len(set(li_train_clusters[i]))):
        

[array([1, 1, 0, ..., 0, 2, 1]), array([0, 0, 3, ..., 1, 2, 0]), array([4, 4, 3, ..., 0, 2, 4]), array([ 2,  9, 15, ...,  3, 20, 23])]


In [37]:
df_test_eng.head(5)

,X8_a,X8_b,X8_c,X8_d,X8_e,X8_f,X8_g,X8_h,X8_i,X8_j,...,X90,X91,X93,X94,X95,X96,X98,X99,qty_options,y
ID,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,54,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,67,0.0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,1,0,57,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,58,0.0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,65,0.0
